# Memoria
Este código está diseñado para ilustrar y comparar tres funciones de memoria diferentes utilizadas en sistemas de IA o chatbots: **<span style="color:green">LastMemories</span>**, **<span style="color:green">LastTokens</span>** y **<span style="color:green">RelevantMemories</span>**. Cada una de estas funciones de memoria cumple un propósito distinto en la gestión y recuperación del historial de conversaciones.

## 1.- Setup inicial

### 1.1- Instalar librerías

In [ ]:
#! pip install openai
#! pip install numpy
#! pip install tenacity
#! pip install python-dotenv

### 1.2.- Cargar librerías

In [ ]:
import os
import openai
import numpy
from datetime import datetime
from llm import generate_text, count_tokens
from dotenv import load_dotenv
from IPython.display import display, HTML

### 1.3.- Variables de entorno

In [ ]:
# Load secrets and config from .env file
load_dotenv()

# OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
print("OpenAI API key: {}".format(openai.api_key[:5] + '...' + openai.api_key[-5:]))

# Model endpoint names
gpt35_model = os.getenv("OPENAI_GPT35_MODEL")
gpt35_16k_model = os.getenv("OPENAI_GPT35_16K_MODEL")
gpt4_model = os.getenv("OPENAI_GPT4_MODEL")
print("GPT-3.5-Turbo model: {}".format(gpt35_model))
print("GPT-3.5-Turbo-16k model: {}".format(gpt35_16k_model))
print("GPT-4 model: {}".format(gpt4_model))

### 1.4.- Importar clases a comparar

In [ ]:
from agents.memory.last_memories import LastMemories
from agents.memory.last_tokens import LastTokens
from agents.memory.relevant_memories import RelevantMemories

## 2.- Definición de funciones

## 2.1.- Función para generar una entrada del usuario


In [ ]:
def generate_user_input(model, conversation):
    if len(conversation) == 1:
        prompt = "Inicia la conversación planteando un tema entretenido o haciendo una pregunta tipo {prompt}. Evita temas de IA y tecnología, sé mas creativo. Imagina que estamos teniendo una charla y que compartimos información gradualmente. Evita proporcionar demasiada información en tu primera pregunta y permite que la conversación se desarrolle de manera natural. Evita preguntas sobre gustos personales, juicios, experiencias personales o detalles específicos. Ten en cuenta que el receptor no posee información personal. Responde solo con la prompt."
        response = generate_text(prompt, model=model)
    else:
        prompt = "Genera un prompt para continuar con la conversación, haciendo preguntas adicionales para ahondar en el tema. Y de vez en cuando, cambia a un nuevo tema. La idea es que la conversación sea una especie de entrevista. Evita proporcionar demasiada información en la pregunta y anima a la IA a compartir sus conocimientos sobre algún tema. Evita preguntas sobre gustos personales, juicios, experiencias personales o detalles específicos, el receptor no tiene información personal. Evita temas de IA y tecnología, sé mas creativo y variado al momento de cambiar de tema. Responde solo con la prompt."
        response = generate_text(prompt, model=model, messages=conversation)
    return response

## 2.2.- Función para simular una conversación y mostrarla

In [ ]:
def simulate_and_display_conversation(model):
    conversation = [{"role": "system", "content": "Entrega siempre respuestas completas, pero no tan extensas"}]

    for i in range(15):  # Simular una conversación de 15 turnos
        user_input = generate_user_input(model, conversation)
        response = generate_text(user_input, model=model, messages=conversation, max_tokens=2000)
        
        conversation.append({"role": "user", "content" : user_input})
        conversation.append({"role": "assistant", "content" : response})
            
    # Mostrar la conversación en formato HTML
    display(HTML("<h3>Conversación generada:</h3>"))
    display(HTML('<table><tr><th>Rol</th><th>Texto</th></tr>' + ''.join([f'<tr><td>{d["role"]}</td><td>{d["content"]}</td></tr>' for d in conversation]) + '</table>'))
    
    return conversation

## 3.- Generar conversación y almacenarla en las respectivas clases

In [ ]:
last_memories = LastMemories()
last_tokens = LastTokens()
relevant_memories = RelevantMemories()

In [ ]:
conversation = simulate_and_display_conversation(gpt4_model) #Se pueden utilizar más modelos = gpt35_model, gpt35_16k_model o gpt4_model
for message in conversation:
    last_memories.add_to_memory(message["role"], message["content"])
    last_tokens.add_to_memory(message["role"], message["content"])
    relevant_memories.add_to_memory(message["role"], message["content"])

### Temas tratados en la conversación:

In [ ]:
prompt = "Considera la conversación y genera una lista con los temas tratados en ella. Responde solo con la lista y conceptos generales."
response = generate_text(prompt, model=gpt4_model, messages=conversation, max_tokens = 500)
print(response)

In [ ]:
tokens = 0
for message in conversation:
    tokens += count_tokens(message["content"])
display(HTML(f'<h3>Total de tokens de la conversación: {tokens}</h3>'))

## 4.- Comparación de las funciones de Memoria

### 4.1.- Last Memories

In [ ]:
display(HTML("<h3>Resultado de la obtención de Last Memories :</h3>"))
display(HTML('<table><tr><th>Rol</th><th>Texto</th></tr>' + ''.join([f'<tr><td>{d["role"]}</td><td>{d["content"]}</td></tr>' for d in last_memories.get_last_memories()]) + '</table>'))

### 4.2.- Last Tokens

In [ ]:
display(HTML("<h3>Resultado de la obtención de Last Tokens:</h3>"))
display(HTML('<table><tr><th>Rol</th><th>Texto</th></tr>' + ''.join([f'<tr><td>{b["role"]}</td><td>{b["content"]}</td></tr>' for b in last_tokens.get_last_tokens()]) + '</table>'))

### 4.3.- Relevant Memories

In [ ]:
#Definiendo la query
prompt = "Considera la conversación y selecciona un concepto cualquiera abordado en ella. Retorna sólo el concepto."
query = generate_text(prompt, model=gpt4_model, messages=conversation)
print(query)

In [ ]:
display(HTML("<h3>Resultado de la obtención de Relevant Memories basado en una query:</h3>"))
display(HTML('<table><tr><th>Rol</th><th>Texto</th></tr>' + ''.join([f'<tr><td>{d["role"]}</td><td>{d["content"]}</td></tr>' for d in relevant_memories.get_relevant_memories(query)]) + '</table>'))